In [1]:
import pandas as pd

data = pd.read_csv('ctr_data_20000.csv')

data.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [2]:
data.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [3]:
data = data.drop(['id','hour','device_id','device_ip'], axis=1)

In [4]:
data.columns

Index(['click', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category',
       'app_id', 'app_domain', 'app_category', 'device_model', 'device_type',
       'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21'],
      dtype='object')

In [5]:
X = data.loc[:, data.columns != 'click'].astype('str')
y = data.loc[:, data.columns == 'click'].astype('int').values.ravel()

In [6]:
X_dic = X.to_dict('records')

X_dic[0]

{'C1': '1005',
 'banner_pos': '0',
 'site_id': '1fbe01fe',
 'site_domain': 'f3845767',
 'site_category': '28905ebd',
 'app_id': 'ecad2386',
 'app_domain': '7801e8d9',
 'app_category': '07d7df22',
 'device_model': '44956a24',
 'device_type': '1',
 'device_conn_type': '2',
 'C14': '15706',
 'C15': '320',
 'C16': '50',
 'C17': '1722',
 'C18': '0',
 'C19': '35',
 'C20': '-1',
 'C21': '79'}

In [7]:
from sklearn.feature_extraction import DictVectorizer

one_hot_encoder = DictVectorizer()

onehot_X = one_hot_encoder.fit_transform(X_dic)

print(X.shape)
print(onehot_X.shape)

(20000, 19)
(20000, 3572)


In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

train_X, test_X = onehot_X[0:15000], onehot_X[15000:]
train_y, test_y = y[0:15000], y[15000:]

decision_tree = DecisionTreeClassifier(criterion='gini', min_samples_split=30)

parameters = {'max_depth': [3, 10, None]}

grid_search = GridSearchCV(decision_tree, parameters, cv=3, scoring='accuracy')

grid_search.fit(train_X, train_y)

print(grid_search.best_params_)

{'max_depth': 3}


In [9]:
from sklearn.metrics import classification_report

model_best = grid_search.best_estimator_
prediction = model_best.predict(test_X)

print(classification_report(test_y, prediction))

              precision    recall  f1-score   support

           0       0.85      0.97      0.91      4141
           1       0.53      0.16      0.25       859

    accuracy                           0.83      5000
   macro avg       0.69      0.57      0.58      5000
weighted avg       0.79      0.83      0.79      5000

